In [1]:
f = open('input.txt')
raw = f.read()[:-1]
f.close()

In [7]:
sample_data = '''.#.
..#
###'''.split('\n')

In [4]:
data = raw.split('\n')

In [148]:
class EnergySource:
    def __init__(self, initial_state_array):
        self.initial_state = initial_state_array
        self.grid = {'0': self.initial_state}
    
    def active_neighbours(self, x, y, z):
        #return number of active neighbours
        ans = 0
        for dz in [-1,0,1]:
            for dy in [-1,0,1]:
                for dx in [-1,0,1]:
                    if (dx == 0) & (dy == 0) & (dz == 0):
                        pass
                    elif (x + dx >= 0) & (y + dy >= 0):
                        try:
                            symbol = self.grid[str(z + dz)][y + dy][x + dx]
                            if symbol == '#':
                                ans += 1
                        except (KeyError, IndexError):
                            pass
        return ans

#If a cube is active and exactly 2 or 3 of its neighbors are also active, the cube remains active. 
#Otherwise, the cube becomes inactive.

#If a cube is inactive but exactly 3 of its neighbors are active, the cube becomes active. 
#Otherwise, the cube remains inactive.
    
    def new_charge(self, x, y, z):
        z = int(z)
        symbol = self.grid[str(z)][y][x]
        if symbol == '#':
            if self.active_neighbours(x, y, z) not in [2, 3]:
                symbol = '.'
        elif symbol == '.':
            if self.active_neighbours(x, y, z) == 3:
                symbol = '#'
        else:
            print(f'Unknown symbol: {symbol}')
            symbol = None
        return symbol

    
    def boot(self, cycles):
        # extend space
        replacement = []
        for line in self.grid['0']:
            replacement.append('.' * cycles + line + '.' * cycles)
            self.grid['0'] = replacement
        self.grid['0'] = ['.' * len(self.grid['0'][0])] * cycles + self.grid['0'] + ['.' * len(self.grid['0'][0])] * cycles
        
        for i in range(1, cycles + 1):
            self.grid[str(i)] = self.grid[str(-i)] = ['.' * len(self.grid['0'][0])] * len(self.grid['0'])
            
        for _ in range(cycles):
            new_grid = {}
            for z in self.grid:
                new_array = []
                for y, line in enumerate(self.grid[z]):
                    new_line = ''
                    for x, _ in enumerate(line):
                        new_line += self.new_charge(x, y, z)
                    new_array.append(new_line)
                new_grid[z] = new_array
            self.grid = new_grid
    
    def active_cubes(self):
        ans = 0
        for z in self.grid:
            for line in self.grid[z]:
                for symbol in line:
                    if symbol == '#':
                        ans += 1
        return ans

In [159]:
test = EnergySource(sample_data)
test.boot(6)
print(test.active_cubes() == 112)

True


In [161]:
%%time
solve = EnergySource(data)
solve.boot(6)
print(solve.active_cubes())

211
CPU times: user 646 ms, sys: 2.31 ms, total: 649 ms
Wall time: 647 ms


In [194]:
class HyperEnergySource:
    def __init__(self, initial_state_array):
        self.initial_state = initial_state_array
        self.grid = {'0': {'0': self.initial_state}}
        
    def __repr__(self):
        for w in self.grid:
            for z in self.grid[w]:
                print(f'w = {w}, z = {z}')
                for line in self.grid[w][z]:
                    print(line)
    
    def active_neighbours(self, x, y, z, w):
        ans = 0
        for dw in [-1,0,1]:
            for dz in [-1,0,1]:
                for dy in [-1,0,1]:
                    for dx in [-1,0,1]:
                        if (dx == 0) & (dy == 0) & (dz == 0) & (dw == 0):
                            pass
                        elif (x + dx >= 0) & (y + dy >= 0):
                            try:
                                symbol = self.grid[str(w + dw)][str(z + dz)][y + dy][x + dx]
                                if symbol == '#':
                                    ans += 1
                            except (KeyError, IndexError):
                                pass
        return ans
    
    def new_charge(self, x, y, z, w):
        z = int(z)
        w = int(w)
        symbol = self.grid[str(w)][str(z)][y][x]
        if symbol == '#':
            if self.active_neighbours(x, y, z, w) not in [2, 3]:
                symbol = '.'
        elif symbol == '.':
            if self.active_neighbours(x, y, z, w) == 3:
                symbol = '#'
        else:
            print(f'Unknown symbol: {symbol}')
            symbol = None
        return symbol
   
    def boot(self, cycles):
        # extend space
        replacement = []
        for line in self.grid['0']['0']:
            replacement.append('.' * cycles + line + '.' * cycles)
        self.grid['0']['0'] = replacement
        self.grid['0']['0'] = ['.' * len(self.grid['0']['0'][0])] * cycles \
                               + self.grid['0']['0'] + ['.' * len(self.grid['0']['0'][0])] * cycles
        
        x_length = len(self.grid['0']['0'][0])
        y_length = len(self.grid['0']['0'])
        
        for i in range(cycles + 1):
            if i != 0:
                self.grid[str(i)] = {}
                self.grid[str(-i)] = {}
            for j in range(cycles + 1):
                if (i == 0) & (j == 0):
                    pass
                else:
                    self.grid[str(i)][str(j)] = ['.' * x_length] * y_length
                    self.grid[str(i)][str(-j)] = ['.' * x_length] * y_length
                    self.grid[str(-i)][str(j)] = ['.' * x_length] * y_length
                    self.grid[str(-i)][str(-j)] = ['.' * x_length] * y_length

        # run cycles   
        for _ in range(cycles):
            new_hyper_grid = {}
            for w in self.grid:
                new_grid = {}
                for z in self.grid[w]:
                    new_array = []
                    for y, line in enumerate(self.grid[w][z]):
                        new_line = ''
                        for x, _ in enumerate(line):
                            new_line += self.new_charge(x, y, z, w)
                        new_array.append(new_line)
                    new_grid[z] = new_array    
                new_hyper_grid[w] = new_grid
            self.grid = new_hyper_grid
    
    def active_cubes(self):
        ans = 0
        for w in self.grid:
            for z in self.grid[w]:
                for line in self.grid[w][z]:
                    for symbol in line:
                        if symbol == '#':
                            ans += 1
        return ans

In [197]:
%%time
test = HyperEnergySource(sample_data)
test.boot(6)
print(test.active_cubes() == 848, '\n')

True 

CPU times: user 19.5 s, sys: 9.49 ms, total: 19.5 s
Wall time: 19.6 s


In [199]:
%%time
solve = HyperEnergySource(data)
solve.boot(6)
print(solve.active_cubes())

1952
CPU times: user 35.3 s, sys: 17.5 ms, total: 35.3 s
Wall time: 35.3 s
